In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def LS(X,Y): #no weight, assumes iid observations
    N = X.shape[0]
    H = np.concatenate((X,np.ones((N,1))), axis=1)
    return np.linalg.inv(H.T@H)@H.T@Y

In [ ]:
def error(x,y,a,b):
    return y-(a*x+b)

In [ ]:
def RANSAC(X,Y,n,k,t,d):
    XY = np.concatenate((X,Y),axis=1)
    N = XY.shape[0]
    #n : nombre de points pour l'ajustement du modele (toujours prendre le minimum ?)
    #k : nb d'iterations
    #t : seuil pour verifier si une donnee matche le modele
    #d : nb points sous le seuil pour que modele corresponde aux donnees
    i = 0
    while(i<k):
        #print(i)
        XYk = XY[np.random.choice(XY.shape[0], size=n, replace=False), :]
        ab = LS(XYk[:,0:1],XYk[:,1:2])
        inliers = np.empty((0,2))
        for j in range(N):
            if np.linalg.norm(error(XY[j:j+1,:1],XY[j:j+1,1:],ab[0,0],ab[1,0])) <= t:
                inliers = np.concatenate((inliers,XY[j:j+1,:]))
        if inliers.shape[0] >= d: #we found a consensus
            ab = LS(inliers[:,0:1],inliers[:,1:2])
            print("RANSAC ended at",i+1 ,"th iteration")
            return ab
        i+=1
    print("RANSAC did not find parameters")

In [ ]:
def getInliersOutliers(X,Y,a,b,t):
    XY = np.concatenate((X,Y),axis=1)
    N = X.shape[0]
    inliers = np.empty((0,2))
    outliers = np.empty((0,2))
    for i in range(N):
        if np.linalg.norm(error(X[i,0],Y[i,0],a,b))<=t:
            inliers = np.concatenate((inliers,XY[i:i+1,:]))
        else:
            outliers = np.concatenate((outliers, XY[i:i+1,:]))
    return (inliers,outliers)

In [ ]:
#y = ax + b + epsilon, epsilon sim Gaussian(0,sigma**2), all epsilon are iid.

#set of inliers
a1 = 2 #model parameter
b1 = 1 #model parameter
N1 = 90 #number of samples
sigma1 = 0.1 #noise std dev

#set of outliers (follow different model, why not the same with high std-dev...)
a2 = 0 #model parameter
b2 = 2 #model parameter
N2 = 10 #number of samples
sigma2 = 0.5 #noise std dev

#RANSAC parameters
#k = 3 #maximum number of iterations : why not compute probability to have a pair of inliers 99% of time with k iterations ? (see below)
t = 3*sigma1 #threshold : absolute value of error between model and sample must be below it to have a sample considered as inlier
d = 0.99*N1 #minimum number of samples considered as inliers to consider the model as OK
n = 2 # number of samples used to build a model, should always be set to minimum (2 here) ?

#probability to have a pair of different inliers :
p = N1/(N1+N2)*(N1-1)/(N1-1+N2) #cf loi hypergeometrique
#number of trials so 99% of time we draw a pair of inliers : Binomiale(X>=1) = 1-Binomiale(X=0)
k = np.log(1-0.99)/np.log(1-p)
#upper rounding (more than 99% chance to draw a pair of inliers)
k = np.ceil(k)

#measurement, ransac, plot, ...
R1 = sigma1**2*np.eye(N1)
R2 = sigma2**2*np.eye(N2)
X1 = np.random.rand(N1,1)
X2 = np.random.rand(N2,1)
Y1 = a1*X1+b1 +  np.linalg.cholesky(R1)@np.random.randn(N1,1)
Y2 = a2*X2+b2 +  np.linalg.cholesky(R2)@np.random.randn(N2,1)
(X,Y) = np.concatenate((X1,X2)),np.concatenate((Y1,Y2))

ransacCandidate = RANSAC(X,Y, n=n, k=k, t=t, d=d)
aRansac = ransacCandidate[0,0]
bRansac = ransacCandidate[1,0]

x = np.arange(0, 1, 0.01)
y = aRansac*x+bRansac
fig, ax = plt.subplots()
ax.plot(x, y,'r')
(inliers,outliers) = getInliersOutliers(X,Y,aRansac,bRansac,t)
plt.scatter(inliers[:,0],inliers[:,1])
plt.scatter(outliers[:,0],outliers[:,1])
print("a = ", aRansac,", b = ", bRansac)